In [5]:
import tensorflow as tf
from tensorflow.keras import layers
import matplotlib.pyplot as plt

# Load MNIST dataset (we'll use digits "1" and "7" as two domains)
(train_images, train_labels), (_, _) = tf.keras.datasets.mnist.load_data()

# Filter only digits "1" and "7"
domain_X = train_images[train_labels == 1]  # Digit 1
domain_Y = train_images[train_labels == 7]  # Digit 7

# Normalize the images
domain_X = (domain_X.astype('float32') - 127.5) / 127.5
domain_X = domain_X.reshape(domain_X.shape[0], 28, 28, 1)

domain_Y = (domain_Y.astype('float32') - 127.5) / 127.5
domain_Y = domain_Y.reshape(domain_Y.shape[0], 28, 28, 1)

# Generator Model (transform between domains)
def build_generator():
    model = tf.keras.Sequential([
        layers.Conv2D(64, (3,3), padding='same', input_shape=(28, 28, 1)),
        layers.LeakyReLU(),
        layers.Conv2D(128, (3,3), padding='same'),
        layers.LeakyReLU(),
        layers.Conv2DTranspose(64, (3,3), strides=2, padding='same'),
        layers.LeakyReLU(),
        layers.Conv2DTranspose(1, (3,3), padding='same', activation='tanh')
    ])
    return model

# Discriminator Model (distinguish between real and fake images)
# Discriminator Model (distinguish between real and fake images)
def build_discriminator():
    model = tf.keras.Sequential([
        layers.Conv2D(64, (3,3), strides=2, padding='same', input_shape=(28, 28, 1)),  # Added padding='same'
        layers.LeakyReLU(),
        layers.Conv2D(128, (3,3), strides=2, padding='same'),  # Added padding='same'
        layers.LeakyReLU(),
        layers.Flatten(),
        layers.Dense(1)
    ])
    return model

# CycleGAN Losses
def cycle_loss(real_image, cycled_image, lambda_cycle=10):
    return lambda_cycle * tf.reduce_mean(tf.abs(real_image - cycled_image))

def generator_loss(fake_output):
    return tf.keras.losses.BinaryCrossentropy()(tf.ones_like(fake_output), fake_output)

def discriminator_loss(real_output, fake_output):
    real_loss = tf.keras.losses.BinaryCrossentropy()(tf.ones_like(real_output), real_output)
    fake_loss = tf.keras.losses.BinaryCrossentropy()(tf.zeros_like(fake_output), fake_output)
    return real_loss + fake_loss

# Training Step for CycleGAN
def train_cyclegan(G, F, D_X, D_Y, epochs=50, batch_size=64, lambda_cycle=10):
    generator_optimizer = tf.keras.optimizers.Adam(1e-4)
    discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

    for epoch in range(epochs):
        for i in range(min(domain_X.shape[0], domain_Y.shape[0]) // batch_size):
            # Get mini-batch of data from both domains
            real_X = domain_X[i*batch_size:(i+1)*batch_size]
            real_Y = domain_Y[i*batch_size:(i+1)*batch_size]

            # Generate fake images
            fake_Y = G(real_X)
            fake_X = F(real_Y)

            # Cycled images
            cycled_X = F(fake_Y)
            cycled_Y = G(fake_X)

            # Train discriminators
            with tf.GradientTape() as tape:
                real_X_output = D_X(real_X)
                fake_X_output = D_X(fake_X)
                real_Y_output = D_Y(real_Y)
                fake_Y_output = D_Y(fake_Y)

                d_X_loss = discriminator_loss(real_X_output, fake_X_output)
                d_Y_loss = discriminator_loss(real_Y_output, fake_Y_output)

            d_X_gradients = tape.gradient(d_X_loss, D_X.trainable_variables)
            d_Y_gradients = tape.gradient(d_Y_loss, D_Y.trainable_variables)

            discriminator_optimizer.apply_gradients(zip(d_X_gradients, D_X.trainable_variables))
            discriminator_optimizer.apply_gradients(zip(d_Y_gradients, D_Y.trainable_variables))

            # Train generators
            with tf.GradientTape() as tape:
                fake_Y = G(real_X)
                fake_X = F(real_Y)

                real_X_output = D_X(real_X)
                real_Y_output = D_Y(real_Y)
                fake_X_output = D_X(fake_X)
                fake_Y_output = D_Y(fake_Y)

                g_loss = generator_loss(fake_Y_output) + cycle_loss(real_X, cycled_X, lambda_cycle)
                f_loss = generator_loss(fake_X_output) + cycle_loss(real_Y, cycled_Y, lambda_cycle)

            g_gradients = tape.gradient(g_loss, G.trainable_variables)
            f_gradients = tape.gradient(f_loss, F.trainable_variables)

            generator_optimizer.apply_gradients(zip(g_gradients, G.trainable_variables))
            generator_optimizer.apply_gradients(zip(f_gradients, F.trainable_variables))

        print(f'Epoch: {epoch+1}, D_X Loss: {d_X_loss}, D_Y Loss: {d_Y_loss}, G Loss: {g_loss}, F Loss: {f_loss}')

    # Generate transformed images
    noise_X = domain_X[:16]
    noise_Y = domain_Y[:16]

    generated_Y = G(noise_X)
    generated_X = F(noise_Y)

    # Plot images
    for i in range(16):
        plt.subplot(4, 4, i+1)
        if i < 8:
            plt.imshow(generated_Y[i, :, :, 0] * 127.5 + 127.5, cmap="gray")
        else:
            plt.imshow(generated_X[i-8, :, :, 0] * 127.5 + 127.5, cmap="gray")
        plt.axis('off')
    plt.show()

# Build models
G = build_generator()  # X to Y
F = build_generator()  # Y to X
D_X = build_discriminator()  # Discriminator for domain X
D_Y = build_discriminator()  # Discriminator for domain Y

# Train CycleGAN
train_cyclegan(G, F, D_X, D_Y)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


ValueError: Exception encountered when calling Sequential.call().

[1mInput 0 of layer "dense_8" is incompatible with the layer: expected axis -1 of input shape to have value 6272, but received input with shape (64, 25088)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(64, 56, 56, 1), dtype=float32)
  • training=None
  • mask=None

In [9]:
# cGAN_mnist.py
import tensorflow as tf
from tensorflow.keras import layers
import matplotlib.pyplot as plt

# Load MNIST dataset
(train_images, train_labels), (_, _) = tf.keras.datasets.mnist.load_data()
train_images = (train_images.astype('float32') - 127.5) / 127.5
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1)

# Generator Model
def build_generator():
    model = tf.keras.Sequential([
        layers.Dense(256, input_shape=(110,)),
        layers.BatchNormalization(),
        layers.LeakyReLU(),
        layers.Dense(512),
        layers.BatchNormalization(),
        layers.LeakyReLU(),
        layers.Dense(1024),
        layers.BatchNormalization(),
        layers.LeakyReLU(),
        layers.Dense(28*28, activation="tanh"),
        layers.Reshape((28, 28, 1))
    ])
    return model

# Discriminator Model
def build_discriminator():
    model = tf.keras.Sequential([
        layers.Flatten(input_shape=(28, 28, 11)),
        layers.Dense(512),
        layers.LeakyReLU(),
        layers.Dense(256),
        layers.LeakyReLU(),
        layers.Dense(1, activation='sigmoid')
    ])
    return model

# Concatenate noise with label embeddings
def concatenate_label_and_noise(labels, noise):
    label_embeddings = tf.one_hot(labels, depth=10)
    noise_with_labels = tf.concat([noise, label_embeddings], axis=1)
    return noise_with_labels

# Conditional GAN training step
class cGAN(tf.keras.Model):
    def __init__(self, generator, discriminator, noise_dim):
        super(cGAN, self).__init__()
        self.generator = generator
        self.discriminator = discriminator
        self.noise_dim = noise_dim

    def compile(self, generator_optimizer, discriminator_optimizer):
        super(cGAN, self).compile()
        self.generator_optimizer = generator_optimizer
        self.discriminator_optimizer = discriminator_optimizer

    def train_step(self, data):
        real_images, labels = data
        batch_size = tf.shape(real_images)[0]
        noise = tf.random.normal([batch_size, self.noise_dim])
        noise_with_labels = concatenate_label_and_noise(labels, noise)

        # Train Discriminator
        with tf.GradientTape() as tape:
            generated_images = self.generator(noise_with_labels)
            real_input = tf.concat([real_images, tf.one_hot(labels, 10)], axis=-1)
            fake_input = tf.concat([generated_images, tf.one_hot(labels, 10)], axis=-1)

            real_output = self.discriminator(real_input)
            fake_output = self.discriminator(fake_input)

            d_loss = tf.keras.losses.BinaryCrossentropy()(tf.ones_like(real_output), real_output) + \
                     tf.keras.losses.BinaryCrossentropy()(tf.zeros_like(fake_output), fake_output)
        gradients = tape.gradient(d_loss, self.discriminator.trainable_variables)
        self.discriminator_optimizer.apply_gradients(zip(gradients, self.discriminator.trainable_variables))

        # Train Generator
        noise_with_labels = concatenate_label_and_noise(labels, noise)
        with tf.GradientTape() as tape:
            generated_images = self.generator(noise_with_labels)
            fake_input = tf.concat([generated_images, tf.one_hot(labels, 10)], axis=-1)
            fake_output = self.discriminator(fake_input)
            g_loss = tf.keras.losses.BinaryCrossentropy()(tf.ones_like(fake_output), fake_output)
        gradients = tape.gradient(g_loss, self.generator.trainable_variables)
        self.generator_optimizer.apply_gradients(zip(gradients, self.generator.trainable_variables))

        return {"d_loss": d_loss, "g_loss": g_loss}

# Instantiate and Train cGAN
generator = build_generator()
discriminator = build_discriminator()
cgan = cGAN(generator, discriminator, noise_dim=100)

cgan.compile(
    generator_optimizer=tf.keras.optimizers.Adam(1e-4),
    discriminator_optimizer=tf.keras.optimizers.Adam(1e-4)
)

cgan.fit((train_images, train_labels), epochs=50, batch_size=64)

# Generate Conditional Images after Training
noise = tf.random.normal([16, 100])
labels = tf.constant([i for i in range(10)] * 2)
noise_with_labels = concatenate_label_and_noise(labels, noise)
generated_images = generator(noise_with_labels)

for i in range(16):
    plt.subplot(4, 4, i+1)
    plt.imshow(generated_images[i, :, :, 0] * 127.5 + 127.5, cmap="gray")
    plt.axis('off')
plt.show()


Epoch 1/50


ValueError: not enough values to unpack (expected 2, got 1)